In [1]:
#query2 SQL API

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1246,application_1732639283265_1209,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import substring
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import count, when, length
import time

spark = SparkSession \
    .builder \
    .appName("DF query 2 execution") \
    .getOrCreate()

# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3006,application_1732639283265_2964,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

In [3]:
#WE USE DATAFRAMES FOR SOME DATA PREPROCESSING SINCE IN ORDER TO RUN SQL API

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
victim19 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header = True
)

victim20 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header = True
)

victim_df19 = victim19.select("AREA ", "Status", "DATE OCC")

victim_df20 = victim20.select("AREA", "Status", "DATE OCC")
victim_df20 = victim_df20.withColumnRenamed("AREA", "AREA ")


# Combine the two datasets
victim_df = victim_df19.union(victim_df20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
police_station = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv",
    header = True
)

police_station.show(3)
victim_df = victim_df.withColumn(
    "AREA ",
    when(col("AREA ") == "01", 1)
    .when(col("AREA ") == "02", 2)
    .when(col("AREA ") == "03", 3)
    .when(col("AREA ") == "04", 4)
    .when(col("AREA ") == "05", 5)
    .when(col("AREA ") == "06", 6)
    .when(col("AREA ") == "07", 7)
    .when(col("AREA ") == "08", 8)
    .when(col("AREA ") == "09", 9)
    .when(col("AREA ") == "10", 10)
    .when(col("AREA ") == "11", 11)
    .when(col("AREA ") == "12", 12)
    .when(col("AREA ") == "13", 13)
    .when(col("AREA ") == "14", 14)
    .when(col("AREA ") == "15", 15)
    .when(col("AREA ") == "16", 16)
    .when(col("AREA ") == "17", 17)
    .when(col("AREA ") == "18", 18)
    .when(col("AREA ") == "19", 19)
    .when(col("AREA ") == "20", 20)
    .when(col("AREA ") == "21", 21)
)
victim_df = victim_df.withColumnRenamed("AREA ", "AREA")
victim_df = victim_df.withColumn(
    "YEAR", 
    when(col("DATE OCC").isNotNull() & (length("DATE OCC") >= 7), substring("DATE OCC", 7, 4)).otherwise(None)
)
police_station = police_station.withColumn(
    "PREC",
    when(col("PREC") == "1", 1)
    .when(col("PREC") == "2", 2)
    .when(col("PREC") == "3", 3)
    .when(col("PREC") == "4", 4)
    .when(col("PREC") == "5", 5)
    .when(col("PREC") == "6", 6)
    .when(col("PREC") == "7", 7)
    .when(col("PREC") == "8", 8)
    .when(col("PREC") == "9", 9)
    .when(col("PREC") == "10", 10)
    .when(col("PREC") == "11", 11)
    .when(col("PREC") == "12", 12)
    .when(col("PREC") == "13", 13)
    .when(col("PREC") == "14", 14)
    .when(col("PREC") == "15", 15)
    .when(col("PREC") == "16", 16)
    .when(col("PREC") == "17", 17)
    .when(col("PREC") == "18", 18)
    .when(col("PREC") == "19", 19)
    .when(col("PREC") == "20", 20)
    .when(col("PREC") == "21", 21)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------------+---+-----------+--------------------+----+
|             X|               Y|FID|   DIVISION|            LOCATION|PREC|
+--------------+----------------+---+-----------+--------------------+----+
|-118.289241553|33.7576608970001|  1|     HARBOR|2175 JOHN S. GIBS...|   5|
|-118.275394206|33.9386273800001|  2|  SOUTHEAST|    145 W. 108TH ST.|  18|
|-118.277669655|33.9703073800001|  3|77TH STREET|    7600 S. BROADWAY|  12|
+--------------+----------------+---+-----------+--------------------+----+
only showing top 3 rows

In [5]:
#USE OF SQL API

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
start_time = time.time()

police_station.createOrReplaceTempView("ps")
victim_df.createOrReplaceTempView("vd")

join_query = "SELECT vd.AREA, ps.DIVISION, vd.YEAR, vd.Status \
            FROM vd \
            INNER JOIN ps on ps.PREC==vd.AREA\
            ORDER BY vd.YEAR"
joined_data = spark.sql(join_query)
# joined_data.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
joined_data.createOrReplaceTempView("mid")

results_query = """
SELECT AREA, 
       DIVISION, 
       YEAR, 
       COUNT(CASE WHEN mid.Status != 'VAGGOS' THEN 1 END) AS Total_Cases,
       COUNT(CASE WHEN mid.Status IN ('CC', 'IC', 'UNK') OR mid.Status IS NULL THEN 1 END) AS open_cases
FROM mid
GROUP BY AREA, DIVISION, YEAR
"""

# Execute the query using Spark SQL
results = spark.sql(results_query)

# Show the first 3 rows
# results.show(3)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
results.createOrReplaceTempView("final_data_view")

query = """
WITH ranked_data AS (
    SELECT DIVISION AS precinct, 
           YEAR AS year, 
           100 * (1 - open_cases / Total_Cases) AS closed_case_rate,
           ROW_NUMBER() OVER (PARTITION BY YEAR ORDER BY 100 * (1 - open_cases / Total_Cases) DESC) AS ranking
    FROM final_data_view
)
SELECT year, precinct, closed_case_rate, ranking
FROM ranked_data
ORDER BY year, ranking
"""

final_results = spark.sql(query)
final_results.show()

end_time = time.time()
elapsed_time = end_time - start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------+------------------+-------+
|year|        precinct|  closed_case_rate|ranking|
+----+----------------+------------------+-------+
|2010|         RAMPART| 32.84713448949121|      1|
|2010|         OLYMPIC|31.515289821999083|      2|
|2010|          HARBOR| 29.36028339237341|      3|
|2010|     WEST VALLEY| 25.97907001834071|      4|
|2010|      HOLLENBECK|25.209289765055367|      5|
|2010|       NORTHEAST|24.705993037915142|      6|
|2010|       SOUTHEAST|24.034296028880863|      7|
|2010|          NEWTON|23.422342234223425|      8|
|2010|        VAN NUYS|21.825750817722266|      9|
|2010| NORTH HOLLYWOOD|20.818301803783545|     10|
|2010|       SOUTHWEST| 18.88270610631132|     11|
|2010|       HOLLYWOOD|16.948968512486427|     12|
|2010|         CENTRAL| 16.94654352085082|     13|
|2010|        WILSHIRE|15.355484974346446|     14|
|2010|         PACIFIC| 9.528531506355364|     15|
|2010|         TOPANGA| 6.825284806936183|     16|
|2010|        FOOTHILL| 5.63636

In [9]:
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 22.54 seconds